In [2]:
import pandas as pd
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.preprocessing import MinMaxScaler
from sklearn import svm


# Классификакция

## Предобработка

In [290]:
data = pd.read_csv("../../data/loan_approval_dataset.csv")
data

,loan_id,no_of_dependents,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,loan_status
0,1,2,Graduate,No,9600000,29900000,12,778,2400000,17600000,22700000,8000000,Approved
1,2,0,Not Graduate,Yes,4100000,12200000,8,417,2700000,2200000,8800000,3300000,Rejected
2,3,3,Graduate,No,9100000,29700000,20,506,7100000,4500000,33300000,12800000,Rejected
3,4,3,Graduate,No,8200000,30700000,8,467,18200000,3300000,23300000,7900000,Rejected
4,5,5,Not Graduate,Yes,9800000,24200000,20,382,12400000,8200000,29400000,5000000,Rejected
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4264,4265,5,Graduate,Yes,1000000,2300000,12,317,2800000,500000,3300000,800000,Rejected
4265,4266,0,Not Graduate,Yes,3300000,11300000,20,559,4200000,2900000,11000000,1900000,Approved
4266,4267,2,Not Graduate,No,6500000,23900000,18,457,1200000,12400000,18100000,7300000,Rejected
4267,4268,1,Not Graduate,No,4100000,12800000,8,780,8200000,700000,14100000,5800000,Approved


In [291]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4269 entries, 0 to 4268
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   loan_id                    4269 non-null   int64 
 1    no_of_dependents          4269 non-null   int64 
 2    education                 4269 non-null   object
 3    self_employed             4269 non-null   object
 4    income_annum              4269 non-null   int64 
 5    loan_amount               4269 non-null   int64 
 6    loan_term                 4269 non-null   int64 
 7    cibil_score               4269 non-null   int64 
 8    residential_assets_value  4269 non-null   int64 
 9    commercial_assets_value   4269 non-null   int64 
 10   luxury_assets_value       4269 non-null   int64 
 11   bank_asset_value          4269 non-null   int64 
 12   loan_status               4269 non-null   object
dtypes: int64(10), object(3)
memory usage: 433.7+ KB


* количество иждивенцев
* образование 
* статус занятости
* годовой доход
* величина займа
* срок кредита в годах
* кредитный рейтинг
* имущественные активы
* коммерческие активы


In [292]:
data = data.drop(['loan_id'], axis=1)

In [293]:
data

,no_of_dependents,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,loan_status
0,2,Graduate,No,9600000,29900000,12,778,2400000,17600000,22700000,8000000,Approved
1,0,Not Graduate,Yes,4100000,12200000,8,417,2700000,2200000,8800000,3300000,Rejected
2,3,Graduate,No,9100000,29700000,20,506,7100000,4500000,33300000,12800000,Rejected
3,3,Graduate,No,8200000,30700000,8,467,18200000,3300000,23300000,7900000,Rejected
4,5,Not Graduate,Yes,9800000,24200000,20,382,12400000,8200000,29400000,5000000,Rejected
...,...,...,...,...,...,...,...,...,...,...,...,...
4264,5,Graduate,Yes,1000000,2300000,12,317,2800000,500000,3300000,800000,Rejected
4265,0,Not Graduate,Yes,3300000,11300000,20,559,4200000,2900000,11000000,1900000,Approved
4266,2,Not Graduate,No,6500000,23900000,18,457,1200000,12400000,18100000,7300000,Rejected
4267,1,Not Graduate,No,4100000,12800000,8,780,8200000,700000,14100000,5800000,Approved


In [294]:
data.columns

Index([' no_of_dependents', ' education', ' self_employed', ' income_annum',
       ' loan_amount', ' loan_term', ' cibil_score',
       ' residential_assets_value', ' commercial_assets_value',
       ' luxury_assets_value', ' bank_asset_value', ' loan_status'],
      dtype='object')

In [295]:
renamed_columns = {}
for column in data.columns:
    renamed_columns[column] = column.strip()
    
data = data.rename(columns = renamed_columns)

In [296]:
data['education'] = data['education'].replace(" Graduate", 1)
data['education'] = data['education'].replace(" Not Graduate", 0)
data['self_employed'] = data['self_employed'].replace(" Yes", 1)
data['self_employed'] = data['self_employed'].replace(" No", 0)
data['loan_status'] = data['loan_status'].replace(" Approved",1)
data['loan_status'] = data['loan_status'].replace(" Rejected",0)


In [297]:
data

,no_of_dependents,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,loan_status
0,2,1,0,9600000,29900000,12,778,2400000,17600000,22700000,8000000,1
1,0,0,1,4100000,12200000,8,417,2700000,2200000,8800000,3300000,0
2,3,1,0,9100000,29700000,20,506,7100000,4500000,33300000,12800000,0
3,3,1,0,8200000,30700000,8,467,18200000,3300000,23300000,7900000,0
4,5,0,1,9800000,24200000,20,382,12400000,8200000,29400000,5000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4264,5,1,1,1000000,2300000,12,317,2800000,500000,3300000,800000,0
4265,0,0,1,3300000,11300000,20,559,4200000,2900000,11000000,1900000,1
4266,2,0,0,6500000,23900000,18,457,1200000,12400000,18100000,7300000,0
4267,1,0,0,4100000,12800000,8,780,8200000,700000,14100000,5800000,1


In [298]:
data.duplicated().sum()

0

In [299]:
f = lambda x : x/1000000
col_million = ['income_annum', 'loan_amount', 'residential_assets_value', 'commercial_assets_value', 'luxury_assets_value', 'bank_asset_value']
for column in col_million: 
    data[column] = data[column].transform(f)
data

,no_of_dependents,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,loan_status
0,2,1,0,9.6,29.9,12,778,2.4,17.6,22.7,8.0,1
1,0,0,1,4.1,12.2,8,417,2.7,2.2,8.8,3.3,0
2,3,1,0,9.1,29.7,20,506,7.1,4.5,33.3,12.8,0
3,3,1,0,8.2,30.7,8,467,18.2,3.3,23.3,7.9,0
4,5,0,1,9.8,24.2,20,382,12.4,8.2,29.4,5.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4264,5,1,1,1.0,2.3,12,317,2.8,0.5,3.3,0.8,0
4265,0,0,1,3.3,11.3,20,559,4.2,2.9,11.0,1.9,1
4266,2,0,0,6.5,23.9,18,457,1.2,12.4,18.1,7.3,0
4267,1,0,0,4.1,12.8,8,780,8.2,0.7,14.1,5.8,1


In [300]:
data['loan_status'].value_counts()

loan_status
1    2656
0    1613
Name: count, dtype: int64

In [301]:
#from sklearn.utils import resample
df_0 = resample(data.loc[data['loan_status']==0],
             replace=True,
             n_samples=len(data.loc[data['loan_status']==1]),
             random_state=42)

df = pd.concat([data.loc[data['loan_status']==1], df_0])

In [302]:
df

,no_of_dependents,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,loan_status
0,2,1,0,9.6,29.9,12,778,2.4,17.6,22.7,8.0,1
6,5,1,0,8.7,33.0,4,678,22.5,14.8,29.2,4.3,1
8,0,1,1,0.8,2.2,20,782,1.3,0.8,2.8,0.6,1
10,4,1,1,2.9,11.2,2,547,8.1,4.7,9.5,3.1,1
13,2,1,1,9.1,31.5,14,679,10.8,16.6,20.9,5.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
4084,0,0,1,9.3,29.4,10,351,24.2,6.8,35.8,6.1,0
159,3,1,1,4.0,10.6,10,318,1.5,5.1,8.0,4.9,0
1037,3,1,1,7.8,27.7,6,430,2.0,8.0,26.1,5.3,0
3733,1,1,0,9.4,26.7,10,343,7.6,2.5,28.6,13.2,0


## Выборки

In [303]:
y = pd.DataFrame(df['loan_status'])
x = pd.DataFrame(df.drop(['loan_status'], axis=1))
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [304]:
scaler=MinMaxScaler()
x_train=pd.DataFrame(scaler.fit_transform(x_train))
x_test=pd.DataFrame(scaler.transform(x_test))

# y_train = y_train.values.ravel()

## Классификатор

In [307]:
from sklearn.metrics import confusion_matrix, precision_score, classification_report, accuracy_score, recall_score
import sklearn.metrics as metric

In [311]:
clf = svm.SVC(kernel='poly')
clf.fit(x_train, y_train)

y_pred_svm = clf.predict(x_test)
print(classification_report(y_test['loan_status'], y_pred_svm))

print ('SVM\n', confusion_matrix(y_test['loan_status'],y_pred_svm))
metric.roc_auc_score(y_test['loan_status'], y_pred_svm)

C:\Users\AlinaZ\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


              precision    recall  f1-score   support

           0       0.92      0.99      0.95       804
           1       0.98      0.92      0.95       790

    accuracy                           0.95      1594
   macro avg       0.95      0.95      0.95      1594
weighted avg       0.95      0.95      0.95      1594

SVM
 [[792  12]
 [ 67 723]]


0.9501322501416966

# Регрессия

## Предобработка

In [17]:
data = pd.read_csv("../../data/possum.csv")
data

,case,site,Pop,sex,age,hdlngth,skullw,totlngth,taill,footlgth,earconch,eye,chest,belly
0,1,1,Vic,m,8.0,94.1,60.4,89.0,36.0,74.5,54.5,15.2,28.0,36.0
1,2,1,Vic,f,6.0,92.5,57.6,91.5,36.5,72.5,51.2,16.0,28.5,33.0
2,3,1,Vic,f,6.0,94.0,60.0,95.5,39.0,75.4,51.9,15.5,30.0,34.0
3,4,1,Vic,f,6.0,93.2,57.1,92.0,38.0,76.1,52.2,15.2,28.0,34.0
4,5,1,Vic,f,2.0,91.5,56.3,85.5,36.0,71.0,53.2,15.1,28.5,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,100,7,other,m,1.0,89.5,56.0,81.5,36.5,66.0,46.8,14.8,23.0,27.0
100,101,7,other,m,1.0,88.6,54.7,82.5,39.0,64.4,48.0,14.0,25.0,33.0
101,102,7,other,f,6.0,92.4,55.0,89.0,38.0,63.5,45.4,13.0,25.0,30.0
102,103,7,other,m,4.0,91.5,55.2,82.5,36.5,62.9,45.9,15.4,25.0,29.0


In [7]:
data.columns

Index(['site', 'Pop', 'sex', 'age', 'hdlngth', 'skullw', 'totlngth', 'taill',
       'footlgth', 'earconch', 'eye', 'chest', 'belly'],
      dtype='object')

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   site      104 non-null    int64  
 1   Pop       104 non-null    object 
 2   sex       104 non-null    object 
 3   age       102 non-null    float64
 4   hdlngth   104 non-null    float64
 5   skullw    104 non-null    float64
 6   totlngth  104 non-null    float64
 7   taill     104 non-null    float64
 8   footlgth  103 non-null    float64
 9   earconch  104 non-null    float64
 10  eye       104 non-null    float64
 11  chest     104 non-null    float64
 12  belly     104 non-null    float64
dtypes: float64(10), int64(1), object(2)
memory usage: 10.7+ KB


In [18]:
data = data.drop(['case'], axis=1)

In [19]:
data = data.rename(columns = {"Pop": "pop"})


In [12]:
data

,site,pop,sex,age,hdlngth,skullw,totlngth,taill,footlgth,earconch,eye,chest,belly
0,1,Vic,m,8.0,94.1,60.4,89.0,36.0,74.5,54.5,15.2,28.0,36.0
1,1,Vic,f,6.0,92.5,57.6,91.5,36.5,72.5,51.2,16.0,28.5,33.0
2,1,Vic,f,6.0,94.0,60.0,95.5,39.0,75.4,51.9,15.5,30.0,34.0
3,1,Vic,f,6.0,93.2,57.1,92.0,38.0,76.1,52.2,15.2,28.0,34.0
4,1,Vic,f,2.0,91.5,56.3,85.5,36.0,71.0,53.2,15.1,28.5,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,7,other,m,1.0,89.5,56.0,81.5,36.5,66.0,46.8,14.8,23.0,27.0
100,7,other,m,1.0,88.6,54.7,82.5,39.0,64.4,48.0,14.0,25.0,33.0
101,7,other,f,6.0,92.4,55.0,89.0,38.0,63.5,45.4,13.0,25.0,30.0
102,7,other,m,4.0,91.5,55.2,82.5,36.5,62.9,45.9,15.4,25.0,29.0


In [20]:
data['pop'] = data['pop'].replace("Vic", 1)
data['pop'] = data['pop'].replace("other", 0)


In [29]:
data = data.rename(columns = {"sex": "sex_m"})

In [30]:
data['sex_m'] = data['sex_m'].replace("m", 1)
data['sex_m'] = data['sex_m'].replace("f", 0)


In [37]:
def f(df):
    if df['sex_m'] == 1:
        return 0
    else: return 1

In [40]:
data['sex_f'] = data.apply(f, axis=1)

In [41]:
data

,site,pop,sex_m,sex_f,age,hdlngth,skullw,totlngth,taill,footlgth,earconch,eye,chest,belly
0,1,1,1,0,8.0,94.1,60.4,89.0,36.0,74.5,54.5,15.2,28.0,36.0
1,1,1,0,1,6.0,92.5,57.6,91.5,36.5,72.5,51.2,16.0,28.5,33.0
2,1,1,0,1,6.0,94.0,60.0,95.5,39.0,75.4,51.9,15.5,30.0,34.0
3,1,1,0,1,6.0,93.2,57.1,92.0,38.0,76.1,52.2,15.2,28.0,34.0
4,1,1,0,1,2.0,91.5,56.3,85.5,36.0,71.0,53.2,15.1,28.5,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,7,0,1,0,1.0,89.5,56.0,81.5,36.5,66.0,46.8,14.8,23.0,27.0
100,7,0,1,0,1.0,88.6,54.7,82.5,39.0,64.4,48.0,14.0,25.0,33.0
101,7,0,0,1,6.0,92.4,55.0,89.0,38.0,63.5,45.4,13.0,25.0,30.0
102,7,0,1,0,4.0,91.5,55.2,82.5,36.5,62.9,45.9,15.4,25.0,29.0
